<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=583f1ba98920279484dd81cf8c48b4a50f1823e4b468cfa1482a7727d0573ef7
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-26 13:46:04
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15


-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: -16.99 K (-0.12%)
Current PnL: -25.51 L (-16.48%)
CY Booked + Current PnL: -11.22 L (-7.25%)
-------------------
Total profit:  1.57 L
Total loss:  -27.08 L
-------------------
Total Booked + Current PnL: 15.76 L (12.33%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.1%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.14 L (63.69%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.6%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,0.60,-18.75,23.19,0.09,18988.0,-18895.0,81882.0,89.97,41.0,M-SC,2.84,253.0,-1.00,0.58,7.57,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.13,8.97,9.84,19.69,20559.0,17193.0,208935.0,-1.63,56.0,X-MC,2.45,78.0,0.84,1.48,16.37,XY25,NTT,AC
52,NESTLEIND,1377.00,1.21,12.40,8.00,21.40,23868.0,32924.0,298350.0,4.29,63.0,X-LC,5.53,12.0,1.38,2.12,19.87,XY25,NTT,FMCG
50,MASFIN,398.61,-1.51,-3.15,26.02,22.05,24690.0,-3090.0,94890.0,-16.44,50.0,H-SC,6.82,164.0,-0.13,0.67,37.93,XR,ATH,FINANCE
43,ITC,452.00,-0.50,-0.85,11.73,10.78,27950.0,-2038.0,238280.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.34,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TITAGARH,1548.0,5.44,-15.16,71.79,45.74,147493.0,-36726.0,205451.0,13.00,70.0,H-SC,6.16,174.0,-0.25,1.46,33.62,XY24,NTT,ENGINEERING
60,REPCOHOME,880.0,4.55,-9.99,110.88,89.82,285950.0,-28614.0,257891.0,-55.29,59.0,H-SC,5.55,160.0,-0.10,1.83,36.44,XY24,NTT,FINANCE
42,IRCTC,1269.0,4.03,-14.79,79.48,52.93,183200.0,-40010.0,230498.0,-36.86,66.0,H-MC,6.93,118.0,-0.22,1.64,6.92,XY24,BTT,TRAVEL
28,GREENPANEL,537.0,2.44,-37.36,128.50,43.14,147370.0,-68393.0,114685.0,157.11,43.0,M-SC,11.13,240.0,-0.46,0.81,7.88,XY24,NTT,MISC
44,JCHAC,2282.0,2.36,-35.76,55.66,-0.01,45695.0,-45709.0,82096.0,16704.35,38.0,M-SC,12.09,233.0,-1.00,0.58,5.07,OX40N,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJESHEXPO,518.0,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
47,KANSAINER,340.0,-2.96,-21.55,44.64,13.47,94438.0,-58113.0,211554.0,-67.73,55.0,H-SC,6.84,158.0,-0.62,1.50,7.57,XY24,NTT,PAINTS
6,ASIANTILES,137.0,-2.73,0.45,80.86,81.67,76197.0,423.0,94233.0,8488.89,70.0,L-SC,14.10,271.0,0.01,0.67,82.49,XR,NTT,CERAMICS
37,IEX,219.0,-2.50,-7.93,61.43,48.63,113087.0,-15863.0,184091.0,-38.85,37.0,H-SC,13.42,136.0,-0.14,1.31,2.70,XR,NTT,MISC
46,JSWINFRA,342.0,-1.69,-3.20,20.27,16.43,43516.0,-7090.0,214684.0,-17.97,59.0,X-MC,8.32,66.0,-0.16,1.52,26.70,X40N,NTT,REALTY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.73,0.45,80.86,81.67,76197.0,423.0,94233.0,8488.89,70.0,L-SC,14.10,271.0,0.01,0.67,82.49,XR,NTT,CERAMICS
17,COALINDIA,484.83,0.40,-0.66,20.02,19.23,29922.0,-998.0,149462.0,16.96,75.0,L-LC,2.81,179.0,-0.03,1.06,20.47,XY25,ATH,MINING


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.25,-2.29,112.79,107.91,162712.0,-3383.0,144261.0,-20.55,53.0,M-SC,11.73,216.0,-0.02,1.02,7.56,OX40N,ATH,CABLES
77,TTKPRESTIG,770.00,0.60,-18.75,23.19,0.09,18988.0,-18895.0,81882.0,89.97,41.0,M-SC,2.84,253.0,-1.00,0.58,7.57,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-1.55,-19.30,23.94,0.02,33723.0,-33695.0,140864.0,114.84,35.0,M-SC,10.56,234.0,-1.00,1.00,22.53,OX40N,NTT,PAINTS
47,KANSAINER,340.00,-2.96,-21.55,44.64,13.47,94438.0,-58113.0,211554.0,-67.73,55.0,H-SC,6.84,158.0,-0.62,1.50,7.57,XY24,NTT,PAINTS
66,SIS,528.00,2.22,-21.94,55.34,21.26,47965.0,-24358.0,86674.0,2047.67,61.0,H-SC,4.85,166.0,-0.51,0.62,17.11,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,-2.73,0.45,80.86,81.67,76197.0,423.0,94233.0,8488.89,70.0,L-SC,14.10,271.0,0.01,0.67,82.49,XR,NTT,CERAMICS
84,WIPRO,420.0,-0.60,9.28,57.63,72.27,106717.0,15728.0,185176.0,-6.08,63.0,M-LC,4.14,99.0,0.15,1.31,16.03,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.60,9.28,57.63,72.27,106717.0,15728.0,185176.0,-6.08,63.0,M-LC,4.14,99.0,0.15,1.31,16.03,XR,NTT,IT
6,ASIANTILES,137.00,-2.73,0.45,80.86,81.67,76197.0,423.0,94233.0,8488.89,70.0,L-SC,14.10,271.0,0.01,0.67,82.49,XR,NTT,CERAMICS
25,FINCABLES,1641.55,-0.25,-2.29,112.79,107.91,162712.0,-3383.0,144261.0,-20.55,53.0,M-SC,11.73,216.0,-0.02,1.02,7.56,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.07,-4.07,115.50,106.72,136650.0,-5024.0,118312.0,-52.54,41.0,H-SC,4.74,139.0,-0.04,0.84,18.35,AR,ATH,MISC
1,ABB,7934.00,-0.25,-4.56,52.52,45.57,131140.0,-11923.0,249696.0,-38.66,53.0,H-MC,4.53,120.0,-0.09,1.77,6.25,AR,NTT,ELECTRICAL


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-0.25,-25.20,105.20,53.50,237451.0,-76037.0,225714.0,-63.76,30.0,X-MC,6.41,58.0,-0.32,1.60,0.25,XY24,NTT,FMCG
9,BAJAJHFL,181.50,0.11,-18.40,90.53,55.47,179722.0,-44764.0,198522.0,-27.60,30.0,X-MC,16.06,64.0,-0.25,1.41,1.72,X40N,ATH,FINANCE
34,HONAUT,58357.33,-0.39,-18.71,76.31,43.32,101034.0,-30472.0,132400.0,-30.16,30.0,X-SC,6.29,90.0,-0.30,0.94,1.66,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.71,-27.42,126.17,64.15,217896.0,-65251.0,172700.0,-57.62,31.0,X-SC,4.55,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
22,DIXON,18931.72,-1.58,-15.90,49.91,26.07,81941.0,-31044.0,164177.0,-59.09,31.0,X-MC,12.51,56.0,-0.38,1.17,2.25,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,0.51,-0.59,21.86,21.14,49401.0,-1349.0,225990.0,-7.66,57.0,X-MC,1.18,74.0,-0.03,1.60,27.38,XY24,NTT,PAINTS
15,CAMS,950.0,-0.29,-0.69,25.35,24.48,63402.0,-1749.0,250107.0,-81.09,48.0,X-SC,1.61,86.0,-0.03,1.78,22.28,X40N,NTT,MISC
78,UNITDSPR,1644.0,0.41,4.90,15.13,20.77,37162.0,11472.0,245616.0,-4.29,50.0,X-MC,1.88,62.0,0.31,1.74,12.16,X40N,NTT,BREWERIES
43,ITC,452.0,-0.50,-0.85,11.73,10.78,27950.0,-2038.0,238280.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.34,X40,NTT,FMCG
20,DABUR,735.0,-0.01,-4.10,49.85,43.70,118589.0,-10176.0,237892.0,-11.99,33.0,X-MC,2.11,72.0,-0.09,1.69,10.33,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.71,-27.42,126.17,64.15,217896.0,-65251.0,172700.0,-57.62,31.0,X-SC,4.55,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
8,AWL,485.00,-0.25,-25.20,105.20,53.50,237451.0,-76037.0,225714.0,-63.76,30.0,X-MC,6.41,58.0,-0.32,1.60,0.25,XY24,NTT,FMCG
19,COLPAL,3726.84,0.42,-20.19,77.30,41.51,162485.0,-53165.0,210200.0,-4.98,40.0,X-MC,8.00,61.0,-0.33,1.49,0.70,XY25,ATH,FMCG
11,BATAINDIA,2096.00,-0.16,-38.84,121.07,35.20,95272.0,-49978.0,78692.0,3.10,35.0,X-SC,17.16,92.0,-0.52,0.56,0.94,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,-0.83,-47.94,192.54,52.31,145514.0,-69584.0,75576.0,-44.54,42.0,X-SC,7.22,91.0,-0.48,0.54,1.16,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.09,-27.57,97.23,42.84,45777.0,-17925.0,47081.0,-52.62,52.0,X-SC,33.31,83.0,-0.39,0.33,6.52,XY24,NTT,MISC
59,RELAXO,1176.00,-0.83,-47.94,192.54,52.31,145514.0,-69584.0,75576.0,-44.54,42.0,X-SC,7.22,91.0,-0.48,0.54,1.16,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.16,-38.84,121.07,35.20,95272.0,-49978.0,78692.0,3.10,35.0,X-SC,17.16,92.0,-0.52,0.56,0.94,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,-0.34,-0.87,25.17,24.08,31675.0,-1107.0,125843.0,-8.86,48.0,X-SC,6.64,89.0,-0.03,0.89,18.57,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.39,-18.71,76.31,43.32,101034.0,-30472.0,132400.0,-30.16,30.0,X-SC,6.29,90.0,-0.30,0.94,1.66,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-1.08,-9.73,33.72,20.71,115131.0,-36805.0,341432.0,-22.33,62.0,X-LC,3.00,1.0,-0.32,2.42,14.09,X40,ATH,IT
41,INFY,1972.00,-0.39,12.79,19.02,34.24,67125.0,40032.0,352920.0,-12.06,65.0,X-LC,6.46,2.0,0.60,2.51,22.14,X40,NTT,IT
75,TMPV,600.00,-1.00,-27.12,68.73,22.98,113159.0,-61255.0,164643.0,-24.30,46.0,X-LC,6.22,3.0,-0.54,1.17,3.55,XY24,NTT,AUTO
81,VBL,671.64,0.83,-2.72,39.46,35.67,121245.0,-8581.0,307261.0,-14.21,56.0,X-LC,3.31,4.0,-0.07,2.18,10.76,X40N,ATH,FMCG
43,ITC,452.00,-0.50,-0.85,11.73,10.78,27950.0,-2038.0,238280.0,-40.74,52.0,X-LC,2.02,5.0,-0.07,1.69,4.34,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.94,-36.80,110.28,32.89,53808.0,-28414.0,48792.0,-702.61,57.0,L-MC,6.92,259.0,-0.53,0.35,34.37,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
50,MASFIN,398.61,-1.51,-3.15,26.02,22.05,24690.0,-3090.0,94890.0,-16.44,50.0,H-SC,6.82,164.0,-0.13,0.67,37.93,XR,ATH,FINANCE
6,ASIANTILES,137.00,-2.73,0.45,80.86,81.67,76197.0,423.0,94233.0,8488.89,70.0,L-SC,14.10,271.0,0.01,0.67,82.49,XR,NTT,CERAMICS
13,BSOFT,831.70,0.07,-18.47,84.95,50.79,97416.0,-25979.0,114674.0,4.75,68.0,H-SC,9.26,151.0,-0.27,0.81,33.52,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.07,-18.47,84.95,50.79,97416.0,-25979.0,114674.0,4.75,68.0,H-SC,9.26,151.0,-0.27,0.81,33.52,XR,ATH,IT
6,ASIANTILES,137.00,-2.73,0.45,80.86,81.67,76197.0,423.0,94233.0,8488.89,70.0,L-SC,14.10,271.0,0.01,0.67,82.49,XR,NTT,CERAMICS
56,RAJESHEXPO,518.00,-5.00,-58.89,143.44,0.07,80882.0,-80790.0,56387.0,1966.92,60.0,L-SC,4.40,268.0,-1.00,0.40,40.24,OX40N,NTT,JEWELLERY
80,VALIANTORG,838.00,-1.23,-42.51,202.91,74.15,190859.0,-69544.0,94061.0,-269.06,53.0,H-SC,21.79,149.0,-0.36,0.67,20.57,XR,NTT,CHEMICALS
17,COALINDIA,484.83,0.40,-0.66,20.02,19.23,29922.0,-998.0,149462.0,16.96,75.0,L-LC,2.81,179.0,-0.03,1.06,20.47,XY25,ATH,MINING


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.44
1,25,44.91
2,50,76.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.02
MC    30.77
LC    24.19
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.90
X40      23.03
X40N     12.90
XR        9.85
XY25      9.17
AR        8.85
OX40N     7.55
SR        0.98
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.18
H-SC    23.90
X-LC    20.72
M-SC    11.50
X-SC     8.07
H-MC     4.84
L-SC     1.55
M-MC     1.40
M-LC     1.31
H-LC     1.10
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.29,-7.12,41.56
IT,13.02,-14.56,72.16
FINANCE,9.67,-16.91,67.76
MISC,7.21,-28.01,80.37
ELECTRICAL,5.94,-12.19,53.28
PAINTS,5.82,-12.55,29.26
INSURANCE,4.68,-2.76,37.93
PHARMA,4.05,-2.94,35.34
AUTO,2.83,-31.71,77.37


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3131309.0,21
AR,1325366.0,10
XR,1274815.0,13
X40,979051.0,14
X40N,869329.0,9
OX40N,726765.0,10
XY25,348504.0,6
SR,281380.0,2
MH,77935.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3610242.0,25
M-SC,1437439.0,15
X-MC,1418185.0,16
X-LC,808365.0,11
X-SC,783262.0,8
H-MC,397190.0,3
L-SC,242848.0,3
M-LC,106717.0,1
H-LC,78457.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1243063.0      6
           AR         906994.0      5
M-SC       XY24       814177.0      6
H-SC       XR         775239.0      7
X-MC       X40        487359.0      7
           XY24       405441.0      3
           X40N       342341.0      4
H-SC       OX40N      325631.0      4
M-SC       OX40N      320252.0      5
X-LC       X40        313728.0      5
X-SC       X40N       309950.0      3
           XY24       295348.0      3
H-SC       SR         281380.0      2
X-LC       X40N       217038.0      2
H-MC       AR         213990.0      2
X-LC       XY24       190080.0      2
H-MC       XY24       183200.0      1
X-MC       XY25       183044.0      2
X-SC       X40        177964.0      2
M-SC       XR         177085.0      2
L-SC       XR         161966.0      2
M-SC       AR         125925.0      2
M-LC       XR         106717.0      1
X-LC       XY25        87519.0      2
L-SC       OX40N       80882.0      1
H-LC       AR          78457.0      1
H-SC       MH          77935.0      1
L-MC       XR          53808.0      1
M-MC       XY25        48019.0      1
L-LC       XY25        29922.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 39.0 seconds
